# scraping

In [1]:
import json
import requests
import numpy as np
import pandas as pd
import time
import calendar
import datetime as dt

### get 1 month of subreddit data:

In [4]:
def get_month_data(month, year, subreddit): # for subsim2015 (<1000 posts/month)
    start=int(dt.datetime(year, month, 1).timestamp())
    end=int(dt.datetime(year, month, calendar.monthrange(year,month)[1]).timestamp())
    uri = 'https://apiv2.pushshift.io/reddit/submission/search/?subreddit={}&limit={}&before={}&after={}'.format(\
             subreddit, 1000, end, start)
    res = requests.get(uri)
    res = pd.DataFrame(res.json()['data'])
    return res
    return res_df

def get_month_data_limited(month, year, subreddit): # for gpt2 (>1000 posts/month)
    res = pd.DataFrame()
    for third in [(1, 10), (11,20), (20, calendar.monthrange(year,month)[1])]:
        start=int(dt.datetime(year, month, third[0]).timestamp())
        end=int(dt.datetime(year, month, third[1]).timestamp())
        uri = 'https://apiv2.pushshift.io/reddit/submission/search/?subreddit={}&limit={}&before={}&after={}'.format(\
                 subreddit, 1000, end, start)
        res_ = requests.get(uri)
        res = pd.concat([res, pd.DataFrame(res_.json()['data'])], sort=False)
        time.sleep(1)
    return res

### get all subreddit data:

In [5]:
def scrape_subreddit(subreddit, start_year, lim=False):
    scrapeddit = pd.DataFrame()
    for year in range(start_year, 2020):
        for month in range(1,13):
            if lim==False: month_data = get_month_data(month, year, subreddit)
            if lim==True: month_data = get_month_data_limited(month, year, subreddit) # for gpt2
            scrapeddit = pd.concat([scrapeddit, month_data], sort=False)
            print(f'{year} {month} added ({len(month_data)} rows)')
            time.sleep(1)
    month_data = get_month_data(1, 2020, subreddit) # add jan 2020
    print(f'2020 1 added ({len(month_data)} rows)')
    return pd.concat([scrapeddit, month_data], sort=False)

---
# `scrape:`

In [6]:
start = time.perf_counter()
subsim = scrape_subreddit('subredditsimulator', 2015)
print(f' completed in {time.perf_counter() - start} seconds')

2015 1 added (0 rows)
2015 2 added (0 rows)
2015 3 added (0 rows)
2015 4 added (0 rows)
2015 5 added (0 rows)
2015 6 added (521 rows)
2015 7 added (688 rows)
2015 8 added (673 rows)
2015 9 added (662 rows)
2015 10 added (524 rows)
2015 11 added (567 rows)
2015 12 added (598 rows)
2016 1 added (594 rows)
2016 2 added (565 rows)
2016 3 added (577 rows)
2016 4 added (628 rows)
2016 5 added (622 rows)
2016 6 added (568 rows)
2016 7 added (604 rows)
2016 8 added (632 rows)
2016 9 added (636 rows)
2016 10 added (662 rows)
2016 11 added (621 rows)
2016 12 added (638 rows)
2017 1 added (624 rows)
2017 2 added (538 rows)
2017 3 added (577 rows)
2017 4 added (574 rows)
2017 5 added (632 rows)
2017 6 added (593 rows)
2017 7 added (662 rows)
2017 8 added (676 rows)
2017 9 added (624 rows)
2017 10 added (658 rows)
2017 11 added (627 rows)
2017 12 added (626 rows)
2018 1 added (619 rows)
2018 2 added (559 rows)
2018 3 added (609 rows)
2018 4 added (599 rows)
2018 5 added (604 rows)
2018 6 added (572

In [7]:
start = time.perf_counter()
gpt2 = scrape_subreddit('subsimulatorgpt2', 2019, lim=True)
print(f' completed in {time.perf_counter() - start} seconds')

2019 1 added (0 rows)
2019 2 added (0 rows)
2019 3 added (0 rows)
2019 4 added (0 rows)
2019 5 added (0 rows)
2019 6 added (1391 rows)
2019 7 added (2061 rows)
2019 8 added (1543 rows)
2019 9 added (1202 rows)
2019 10 added (1385 rows)
2019 11 added (1344 rows)
2019 12 added (1384 rows)
2020 1 added (1000 rows)
 completed in 118.71145535800008 seconds


---
# export

In [11]:
subsim.to_csv('data/ss_data.csv', index=False)
gpt2.to_csv('data/gp_data.csv', index=False)